# MadMiner particle physics tutorial

# Appendix 6: Finite differences instead of morphing

Johann Brehmer, Felix Kling, Irina Espejo, Sinclert Pérez, and Kyle Cranmer 2018-2020

## Preparation

*Important*: You will need the MadMiner version on the `finite-differences` branch (https://github.com/diana-hep/madminer/tree/finite-differences). Get it with `git clone  -branch finite-differences https://github.com/diana-hep/madminer.git` followed by `pip install -e ./madminer`.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline


In [ ]:
# MadMiner output
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [ ]:
from madminer import MadMiner, LHEReader, combine_and_shuffle, plot_distributions, SampleAugmenter, sampling, ScoreEstimator

Please enter here the path to your MG5 root directory and to a Python2.7 executable that can be used to run MadGraph:

In [ ]:
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_7_0'
python2 = '/Users/johannbrehmer/anaconda3/envs/python2/bin/python'

## 1. Input data

We consider Drell-Yan Z production with a decay into two muons (in this tutorial, the photon contribution to that process is turned off).

Have a look at the `cards` folder. You'll find text files ("cards") that specify the process simulation in typical MadGraph convention.

## 2. Define parameter space

After creating a `MadMiner` instance, the first important step is the definition of the parameter space. Each model parameter is characterized by a name as well as the LHA block and ID.

In our case, we're just interested in a single parameter, the Z boson mass:

In [ ]:
miner = MadMiner()
miner.add_parameter(lha_block='mass', lha_id=23, parameter_name='mz')


## 3. Define benchmarks (evaluation points for |M|^2)

The next step is the definition of all the points at which the weights (squared matrix elements) should be evaluated by MadGraph. We call these points "benchmarks" and define them with the `add_benchmark` function. Here we really just need one standard value, for which we use 91.2 GeV:

In [ ]:
miner.add_benchmark({'mz':91.2}, 'mz_91_2')

## 4. Finite difference setup

To construct the optimal observables, we will later need to compute the joint score. To compute the joint score, we need to compute the derivative of the event weights with respect to the parameter of interest (the Z mass). To compute this derivative, we use the method of finite differences, numerically computing the event weights both for `mz = 91.2 GeV` and for `mz = 91.2 GeV + ε` and using

```
d σ(mz) / d mz  ≈  [σ(91.2 GeV + ε) - σ(91.2 GeV)] / ε.
```

MadMiner supports this through the `finite_differences()` function, which adds an additional benchmark at `mz = 91.2 GeV + ε`.

In [ ]:
miner.finite_differences(epsilon=0.1)

## 5. Save setup

The parameter space, benchmark points, and finite-difference setup are saved in a HDF5 file:

In [ ]:
miner.save('data/setup_fd.h5')

They can now be loaded again with `miner.load(filename)`.

## 6. Generate events with MadGraph

To load this saved data (say if you want to repeat the following steps in a later session), you can just use this:

In [ ]:
miner = MadMiner()
miner.load("data/setup_fd.h5")

In a next step, MadMiner calls MadGraph and creates the process folder. You have to provide paths to the process card, run card, param card (the entries corresponding to the parameters of interest will be automatically adapted), and an empty reweight card. Log files in the `log_directory` folder collect the MadGraph output and may be important for debugging.

The `sample_benchmark` keyword determines for which parameter setup we generate events.

One slight annoyance is that MadGraph only supports Python 2. The `run()` and `run_multiple()` commands have a keyword `initial_command` that let you load a virtual environment in which `python` maps to Python 2 (which is what we do below). Alternatively / additionally you can set `python2_override=True`, which calls `python2.7` instead of `python` to start MadGraph.

In [ ]:
miner.run(
    sample_benchmark='mz_91_2',
    mg_directory=mg_dir,
    mg_process_directory='mg_processes/drell-yen',
    proc_card_file='cards/proc_card_dy.dat',
    param_card_template_file='cards/param_card_dy.dat',
    run_card_file='cards/run_card_dy.dat',
    log_directory='logs/drell-yan',
    python_executable=python2,
    only_prepare_script=True
)

## 7. Generate events

Now follow the last printed line in the output of the last cell and run the run.sh script that actually generates the events. This will take a moment.

After running any event generation through MadMiner, you should check whether the run succeeded: are the usual output files there, do the log files show any error messages? MadMiner does not (yet) perform any explicit checks, and if something went wrong in the event generation, it will only notice later when trying to load the event files.

## 8. Prepare analysis of the LHE samples

The `madminer.lhe` submodule allows us to extract observables directly from the parton-level LHE samples, including an approximate description of the detector response with smearing functions. The central object is an instance of the `LHEReader` class, which has to be initialized with a MadMiner file:

In [ ]:
lhe = LHEReader('data/setup_fd.h5')

After creating the `LHEReader` object, one can add a number of event samples (the output of running MadGraph in step 1) with the `add_sample()` function.

In addition, you have to provide the information which sample was generated from which benchmark with the `sampled_from_benchmark` keyword, and set `is_background=True` for all background samples.

In [ ]:
lhe.add_sample(
    lhe_filename='mg_processes/drell-yen/Events/run_01/unweighted_events.lhe.gz',
    sampled_from_benchmark='mz_91_2',
)


## 9. Observables and cuts

The next step is the definition of observables, either through a Python function or an expression that can be evaluated. Here we demonstrate the latter, which is implemented in `add_observable()`. In the expression string, you can use the terms `j[i]`, `e[i]`, `mu[i]`, `a[i]`, `met`, where the indices `i` refer to a ordering by the transverse momentum. In addition, you can use `p[i]`, which denotes the `i`-th particle in the order given in the LHE sample (which is the order in which the final-state particles where defined in MadGraph).

All of these represent objects inheriting from scikit-hep [LorentzVectors](http://scikit-hep.org/api/math.html#vector-classes), see the link for a documentation of their properties. In addition, they have `charge` and `pdg_id` properties.

`add_observable()` has an optional keyword `required`. If `required=True`, we will only keep events where the observable can be parsed, i.e. all involved particles have been detected. If `required=False`, un-parseable observables will be filled with the value of another keyword `default`.

In a realistic project, you would want to add a large number of observables that capture all information in your events. Here we will just define one observable, the invariant mass of the muon pair.

In [ ]:
lhe.add_observable('mll', '(mu[0] + mu[1]).m', required=True,)


We can also add cuts, again in parse-able strings. In addition to the objects discussed above, they can contain the observables:

In [ ]:
lhe.add_cut('mll > 10.')


## 10. Parse events

The function `analyse_samples` then calculates all observables from the LHE file(s) generated before, applies the smearing, and checks which events pass the cuts:

In [ ]:
lhe.analyse_samples()

The values of the observables and the weights are then saved in the HDF5 file. It is possible to overwrite the same file, or to leave the original file intact and save all the data into a new file as follows:

In [ ]:
lhe.save('data/lhe_data_fd.h5')

## 11. Plot distributions

Let's see what our simulation produced:

In [ ]:
_ = plot_distributions(
    filename='data/lhe_data_fd.h5',
    parameter_points=['mz_91_2'],
    uncertainties='none',
    n_bins=50,
    n_cols=3,
    normalize=True,
)

That should look roughly like a Breit-Wigner :)

## 12. Extract training and test data

At this point, we have all the information we need from the simulations. But the data is not quite ready to be used for machine learning. The `madminer.sampling` class `SampleAugmenter` will take care of the remaining book-keeping steps before we can fit a model.

First, it unweights the samples, i.e. for a given parameter vector `theta` (or a distribution `p(theta)`) it picks events `x` such that their distribution follows `p(x|theta)`. The selected samples will all come from the event file we have so far, but their frequency is changed -- some events will appear multiple times, some will disappear.

Second, `SampleAugmenter` calculates the joint score that we will later use as targets in a regression problem.

In [ ]:
sampler = SampleAugmenter('data/lhe_data_fd.h5')

In [ ]:
x, _, t_xz, _ = sampler.sample_train_local(
    theta=sampling.benchmark('mz_91_2'),
    n_samples=100000,
    folder='./data/samples',
    filename='train_score',
    validation_split=0.0,
    test_split=0.2,
)

We can use the same data as in part 3a, so you only have to execute this if you haven't gone through tutorial 3a:

In [ ]:
_ = sampler.sample_test(
    theta=sampling.benchmark('mz_91_2'),
    n_samples=1000,
    folder='./data/samples',
    filename='test',
    validation_split=0.0,
    test_split=0.2,
)

## 13. Plot the training data

Let's have a look at what we just generated:

In [ ]:
fig = plt.figure(figsize=(4,4))

plt.hist(x.flatten(), bins=50, range=(75,100), histtype="step", lw=1.5, density=True)

plt.xlabel(r"$m_{\ell\ell}$")
plt.ylabel(r"Histogram")

plt.show()

In [ ]:
fig = plt.figure(figsize=(4,4))

plt.scatter(x.flatten()[::100], t_xz.flatten()[::100], alpha=0.3, s=3.)

plt.xlabel(r"$x$")
plt.ylabel(r"$t(x,z)$")

plt.xlim(80., 100.)

plt.show()

What's this? This is training data for the optimal observable we want to fit!